In [1]:
import numpy as np
import matplotlib.pyplot as plt

The lagrangian has a quadratic term and zero order term in terms of $\frac{\partial q}{\partial t}$

L = L_2 - L_0

Therefore H = L_2 + L_0

In [6]:
def y_1(x):
    return j

SyntaxError: invalid syntax (<ipython-input-6-14d6a0ed2d7e>, line 2)

In [11]:
complex(8,3)

(8+3j)

In [12]:
L_2 = 1/2*(x_1_dot**2 + y_1_dot**2) + 1/2*(x_2_dot**2 + y_2_dot**2)

L_0 =  ( -1/2 * (complex(x_1,y_1) + Lambda_1*(complex(x_2,y_2))^3 )(complex(x_1,-y_1) + Lambda_1*(complex(x_2,-y_2))^3) 
         -1/2 * (complex(x_2,y_2) + Lambda_2*(complex(x_1,y_1))^3 )(complex(x_2,-y_2) + Lambda_2*(complex(x_1,-y_1))^3) )

# Define Lagrangian
L = L_2 - L_0

NameError: name 'x_1_dot' is not defined

In [13]:
# Define Hamiltonian
H = 1/2*(p_x_1**2 + p_y_1**2) + 1/2*(p_x_2**2 + p_y_2**2) + L_0

NameError: name 'p_x_1' is not defined

In [ ]:
i = 0

# Timestep
delta_t = 0.005


# Initial coordinates 
# old_coordinates = ["x_1", "x_2", "y_1", "y_2", 
                    #"p_x_1", "p_x_2", "p_y_1", "p_y_2", 
                    #"p_x_dot_1", "p_x_dot_2", "p_y_dot_1", "p_y_dot_2"]

old_coordinates = [.5, .2, .3, .4,
                   0, 0, 0, 0,
                   0, 0, 0, 0]

#Coefficients
Lambda_1 = 0.01
Lambda_2 = 0.015

# Coordinates list to record simulation dynamics
coordinates_list = ["x_1", "x_2", "y_1", "y_2", 
                    "p_x_1", "p_x_2", "p_y_1", "p_y_2", 
                    "p_x_dot_1", "p_x_dot_2", "p_y_dot_1", "p_y_dot_2"]

# Lists for recording x and v and a.
for el in coordinates_list:
    el = []

# Repeating algorithm
while i < 5001:
    # b) velocity Verlet 1 to get new positions from old positions, momentums and rate of change of momentums
    
    
    # c) Use 
    
    
    
    # d)
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    new_coordinates = [x_1_new, x_2_new, y_1_new, y_2_new, 
                       p_x_1_new, p_x_2_new, p_y_1_new, p_y_2_new,
                       p_x_dot_1_new, p_x_dot_2_new, p_y_dot_1_new, p_y_dot_2_new]
    
    # Recording position, momentum, and rate of change of momentum every 10th step
    if i % 10 == 0:
        
        h = 0
        
        while h < len(coordinates_list):
            coordinates_list[h] = coordinates_list[h].append(new_coordinates[h])
            h += 1
            
        #recorded_x_1 = recorded_x_1.append(x_1_new)
        #recorded_p_x_1 = recorded_p_x_1.append(p_x_1_new)
        #recorded_p_x_dot_1 = recorded_p_x_dot_1.append(p_x_dot_1_new)

        #recorded_y_1 = recorded_y_1.append(y_1_new)
        #recorded_p_y_1 = recorded_p_y_1.append(p_y_1_new)
        #recorded_p_y_dot_1 = recorded_p_y_dot_1.append(p_y_dot_1_new)

        #recorded_x_2 = recorded_x_2.append(x_2_new)
        #recorded_p_x_2 = recorded_p_x_2.append(p_x_2_new)
        #recorded_p_x_dot_2 = recorded_p_x_dot_2.append(p_x_dot_2_new)

        #recorded_y_2 = recorded_y_2.append(y_2_new)
        #recorded_p_y_2 = recorded_p_y_2.append(p_y_2_new)
        #recorded_p_y_dot_2 = recorded_p_y_dot_2.append(p_y_dot_2_new)
        
    # Update new coordinates
    
    old_coordinates = new_coordinates
    #x_1_old = x_1_new
    #y_1_old = y_1_new
    #x_2_old = x_2_new
    #y_2_old = y_2_new
    
    #p_x_1_old = p_x_1_new
    #p_y_1_old = p_y_1_new
    #p_x_2_old = p_x_2_new
    #p_y_2_old = p_y_2_new

    #p_x_1_dot_old = p_x_1_dot_new
    #p_y_1_dot_old = p_y_1_dot_new
    #p_x_2_dot_old = p_x_2_dot_new
    #p_y_2_dot_old = p_y_2_dot_new

    i += 1